<img src="https://i.imgur.com/6U6q5jQ.png"/>

In [ ]:
import geopandas as gpd

mapGitLink='https://github.com/SocialAnalytics-StrategicIntelligence/GeoDFBasics_py/raw/main/maps/theMapAndData.gpkg'

theMapAndData=gpd.read_file(mapGitLink, layer='fragility')

In [ ]:
theMapAndData.columns

In [ ]:

IndexWide=['COUNTRY', 'iso3','geometry', 'Year']
VarsToWide=['Total_mnmx', 'Total_ei5']
colsForWide=IndexWide+VarsToWide
theMapAndData.loc[:,colsForWide].set_index(IndexWide).unstack(level='Year')


In [ ]:
theMapAndData_wideYears=theMapAndData.loc[:,colsForWide].set_index(IndexWide).unstack(level='Year').reset_index(drop=False)
theMapAndData_wideYears.columns

In [ ]:
theMapAndData_wideYears.columns=['_'.join((a[0],str(a[1]))) if a[1]!='' else a[0] for a in theMapAndData_wideYears.columns]
theMapAndData_wideYears

In [ ]:
theMapAndData_wideYears.columns[theMapAndData_wideYears.columns.str.startswith('Total_m')]

In [ ]:
mnmxCols=theMapAndData_wideYears.columns[theMapAndData_wideYears.columns.str.startswith('Total_m')]

theMapAndData_wideYears['deteriorating_along10']=theMapAndData_wideYears.loc[:,mnmxCols].apply(lambda row: row.is_monotonic_increasing, axis=1)
theMapAndData_wideYears['improving_along10']=theMapAndData_wideYears.loc[:,mnmxCols].apply(lambda row: row.is_monotonic_decreasing, axis=1)
theMapAndData_wideYears['equal_along10']=theMapAndData_wideYears.loc[:,mnmxCols].apply(lambda row: row.std()==0, axis=1)

In [ ]:
theMapAndData_wideYears[theMapAndData_wideYears.deteriorating_along10]

In [ ]:
theMapAndData_wideYears[theMapAndData_wideYears.improving_along10]

In [ ]:
theMapAndData_wideYears[theMapAndData_wideYears.equal_along10]

In [ ]:
theMapAndData_wideYears.columns

In [ ]:
theMapAndData_wideYears['better_after10']=theMapAndData_wideYears.Total_ei5_2013>theMapAndData_wideYears.Total_ei5_2023
theMapAndData_wideYears['worse_after10']=theMapAndData_wideYears.Total_ei5_2013<theMapAndData_wideYears.Total_ei5_2023
theMapAndData_wideYears['same_after10']=theMapAndData_wideYears.Total_ei5_2013==theMapAndData_wideYears.Total_ei5_2023

In [ ]:
SubCols=theMapAndData_wideYears.columns[:2].to_list() + theMapAndData_wideYears.columns[-6::].to_list()
theMapAndData_wideYears.loc[:,SubCols]

In [ ]:
theMapAndData_wideYears['along10']=theMapAndData_wideYears.loc[:,['deteriorating_along10','improving_along10','equal_along10']].idxmax(axis=1)
theMapAndData_wideYears['after10']=theMapAndData_wideYears.loc[:,['better_after10', 'worse_after10','same_after10']].idxmax(axis=1)

In [ ]:
theMapAndData_wideYears

In [ ]:
theMapAndData_wideYears['along10'].value_counts()

In [ ]:
theMapAndData_wideYears['after10'].value_counts()

In [ ]:
theMapAndData_wideYears.info()

In [ ]:
theMapAndData_wideYears=gpd.GeoDataFrame(theMapAndData_wideYears, geometry=theMapAndData_wideYears.geometry, crs="EPSG:4326")
theMapAndData_wideYears.info()

In [ ]:
needed=['COUNTRY','iso3','geometry','after10']
theMapAndData_wideYears.loc[:,needed]

In [ ]:
import os

theMapAndData_wideYears.loc[:,needed].to_file(os.path.join("maps","fragilitymap.gpkg"),layer='after10', driver="GPKG")